In [1]:
import random
import unittest
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

# Глобальный реестр классов
class_registry = {'plants': {}, 'animals': {}}

# Метакласс для растений
class EvalPlantMeta(type):
    def __new__(cls, name, bases, attrs):
        # Регистрация подкласса
        if name not in ('Plant',):  # Пропускаем базовый класс
            class_registry['plants'][name] = {'attrs': attrs}

        # DSL для стратегии роста
        strategy = attrs.get('strategy', {})
        grow_time = strategy.get('grow', {}).get('time', [])
        grow_chance = strategy.get('grow', {}).get('chance', 0.5)

        # Добавляем атрибуты
        attrs['grow_chance'] = grow_chance

        # Генерация метода spread()
        def spread(self, world, time_of_day):
            if time_of_day not in (grow_time if isinstance(grow_time, list) else [grow_time]):
                return []
            neighbors = [(self.x + dx, self.y + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]
            valid_neighbors = []
            for nx, ny in neighbors:
                if 0 <= nx < world.width and 0 <= ny < world.height and not world.grid[nx][ny]:
                    valid_neighbors.append((nx, ny))
            # Адаптация: уменьшаем шанс роста при высокой плотности
            plant_density = sum(1 for e in world.entities if isinstance(e, Plant)) / (world.width * world.height)
            adjusted_chance = max(0.1, self.grow_chance * (1 - plant_density))
            if valid_neighbors and random.random() < adjusted_chance:
                nx, ny = random.choice(valid_neighbors)
                logger.info(f"{self.__class__.__name__} spreads to ({nx}, {ny})")
                return [self.__class__(nx, ny)]
            return []

        attrs['spread'] = spread
        return super().__new__(cls, name, bases, attrs)

# Метакласс для животных
class EvalAnimalMeta(type):
    def __new__(cls, name, bases, attrs):
        # Регистрация подкласса
        if name not in ('Animal',):
            class_registry['animals'][name] = {'attrs': attrs}

        # DSL для стратегии поведения
        strategy = attrs.get('strategy', {})
        active_times = strategy.get('active_times', [])
        eat_targets = strategy.get('eat', {}).get('targets', [])
        eat_chance = strategy.get('eat', {}).get('chance', 0.5)
        move_chance = strategy.get('move', {}).get('chance', 0.5)
        reproduce_chance = strategy.get('reproduce', {}).get('chance', 0.1)

        # Добавляем атрибуты
        attrs['move_chance'] = move_chance
        attrs['eat_chance'] = eat_chance
        attrs['reproduce_chance'] = reproduce_chance

        # Генерация метода eat()
        def eat(self, world, time_of_day):
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = self.x + dx, self.y + dy
                if 0 <= nx < world.width and 0 <= ny < world.height:
                    entity = world.grid[nx][ny]
                    if entity and entity.__class__ in eat_targets:
                        world.grid[nx][ny] = None
                        world.entities.remove(entity)
                        self.hunger = max(0, self.hunger - 3)
                        logger.info(f"{self.__class__.__name__} at ({self.x}, {self.y}) ate {entity.__class__.__name__}")
                        break

        # Генерация метода move()
        def move(self, world):
            neighbors = [(self.x + dx, self.y + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]
            valid_neighbors = [(nx, ny) for nx, ny in neighbors if 0 <= nx < world.width and 0 <= ny < world.height]
            food_density = sum(1 for e in world.entities if e.__class__ in eat_targets) / (world.width * world.height)
            adjusted_move_chance = min(0.9, self.move_chance + (0.5 - food_density))
            if valid_neighbors and random.random() < adjusted_move_chance:
                self.x, self.y = random.choice(valid_neighbors)
                logger.info(f"{self.__class__.__name__} moved to ({self.x}, {self.y})")

        # Генерация метода reproduce()
        def reproduce(self, world):
            if random.random() < self.reproduce_chance and self.hunger < 3:
                neighbors = [(self.x + dx, self.y + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]
                valid_neighbors = [(nx, ny) for nx, ny in neighbors if 0 <= nx < world.width and 0 <= ny < world.height and not world.grid[nx][ny]]
                if valid_neighbors:
                    nx, ny = random.choice(valid_neighbors)
                    logger.info(f"{self.__class__.__name__} reproduced at ({nx}, {ny})")
                    return [self.__class__(nx, ny)]
            return []

        # Генерация метода act()
        def act(self, world, time_of_day):
            self.hunger += 1
            if time_of_day not in active_times:
                return []
            if random.random() < self.eat_chance:
                self.eat(world, time_of_day)
            self.move(world)
            return self.reproduce(world)

        attrs['eat'] = eat
        attrs['move'] = move
        attrs['reproduce'] = reproduce
        attrs['act'] = act
        return super().__new__(cls, name, bases, attrs)

# Базовые классы с метаклассами
class Plant(metaclass=EvalPlantMeta):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.active = False

class Animal(metaclass=EvalAnimalMeta):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.hunger = 0
        self.group_size = 1

# Определения видов с DSL
class Dandelion(Plant):
    strategy = {'grow': {'time': 'day', 'chance': 0.5}}

class Amanita(Plant):
    strategy = {'grow': {'time': 'night', 'chance': 0.4}}

class Orange(Plant):
    strategy = {'grow': {'time': ['morning', 'evening'], 'chance': 0.6}}

class Chimpanzee(Animal):
    strategy = {
        'active_times': ['morning', 'day', 'evening'],
        'eat': {'targets': [Dandelion], 'chance': 0.5},
        'move': {'chance': 0.5},
        'reproduce': {'chance': 0.1}
    }

class Orangutan(Animal):
    strategy = {
        'active_times': ['morning', 'evening'],
        'eat': {'targets': [Orange, Amanita, Chimpanzee], 'chance': 0.7},
        'move': {'chance': 0.3},
        'reproduce': {'chance': 0.05}
    }

class Time:
    def __init__(self):
        self.cycle = 0
        self.times_of_day = ["morning", "day", "evening", "night"]

    def get_time(self):
        return self.times_of_day[self.cycle % 4]

    def tick(self):
        self.cycle += 1

class World:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.grid = [[None for _ in range(height)] for _ in range(width)]
        self.entities = []

    def add_entity(self, entity):
        if not self.grid[entity.x][entity.y]:
            self.grid[entity.x][entity.y] = entity
            self.entities.append(entity)

    def step(self, time_of_day):
        new_entities = []
        for entity in self.entities[:]:
            if isinstance(entity, Plant):
                new_plants = entity.spread(self, time_of_day)
                new_entities.extend(new_plants)
            elif isinstance(entity, Animal):
                new_animals = entity.act(self, time_of_day)
                new_entities.extend(new_animals)
        for entity in new_entities:
            self.add_entity(entity)

    def display(self):
        symbols = {Dandelion: 'D', Amanita: 'A', Orange: 'O', Chimpanzee: 'C', Orangutan: 'An', None: '.'}
        for i in range(self.width):
            row = [symbols[type(self.grid[i][j])] if self.grid[i][j] else '.' for j in range(self.height)]
            print(' '.join(row))
        print()

# Юнит-тесты
class TestEcosystem(unittest.TestCase):
    def setUp(self):
        self.world = World(5, 5)
        self.time = Time()

    def test_class_registration(self):
        self.assertIn('Dandelion', class_registry['plants'])
        self.assertIn('Chimpanzee', class_registry['animals'])
        self.assertEqual(len(class_registry['plants']), 3)
        self.assertEqual(len(class_registry['animals']), 2)

    def test_plant_methods(self):
        dandelion = Dandelion(0, 0)
        self.assertTrue(hasattr(dandelion, 'spread'))
        self.assertEqual(dandelion.grow_chance, 0.5)

    def test_animal_methods(self):
        chimpanzee = Chimpanzee(0, 0)
        self.assertTrue(hasattr(chimpanzee, 'eat'))
        self.assertTrue(hasattr(chimpanzee, 'move'))
        self.assertTrue(hasattr(chimpanzee, 'reproduce'))

    def test_plant_spread(self):
        dandelion = Dandelion(2, 2)
        self.world.add_entity(dandelion)
        new_plants = dandelion.spread(self.world, 'day')
        self.assertTrue(len(new_plants) <= 1)
        new_plants = dandelion.spread(self.world, 'night')
        self.assertEqual(new_plants, [])

    def test_animal_behavior(self):
        chimpanzee = Chimpanzee(2, 2)
        self.world.add_entity(chimpanzee)
        self.world.add_entity(Dandelion(2, 3))
        chimpanzee.act(self.world, 'morning')
        self.assertTrue(chimpanzee.hunger >= 0)

def main():
    world = World(10, 10)
    time = Time()

    for _ in range(10):
        x, y = random.randint(0, 9), random.randint(0, 9)
        world.add_entity(Dandelion(x, y))
        x, y = random.randint(0, 9), random.randint(0, 9)
        world.add_entity(Amanita(x, y))
        x, y = random.randint(0, 9), random.randint(0, 9)
        world.add_entity(Orange(x, y))
    for _ in range(5):
        x, y = random.randint(0, 9), random.randint(0, 9)
        world.add_entity(Chimpanzee(x, y))
        x, y = random.randint(0, 9), random.randint(0, 9)
        world.add_entity(Orangutan(x, y))

    for step in range(20):
        time_of_day = time.get_time()
        logger.info(f"Step {step + 1}, Time: {time_of_day}")
        world.step(time_of_day)
        world.display()
        time.tick()

if __name__ == "__main__":
    # Запуск тестов
    unittest.main(argv=[''], exit=False)
    # Запуск симуляции
    main()

.....
----------------------------------------------------------------------
Ran 5 tests in 0.007s

OK
Step 1, Time: morning
Orange spreads to (2, 5)
Orange spreads to (6, 3)
Orange spreads to (0, 8)
Orangutan at (5, 2) ate Amanita
Orangutan moved to (6, 2)
Orangutan at (7, 2) ate Orange
Orangutan moved to (8, 2)
Chimpanzee moved to (5, 1)
Chimpanzee moved to (6, 6)
Orangutan moved to (8, 4)
Chimpanzee moved to (9, 0)
Orangutan at (4, 2) ate Chimpanzee
Step 2, Time: day
Dandelion spreads to (2, 0)
Dandelion spreads to (6, 1)
Dandelion spreads to (1, 2)
Chimpanzee moved to (7, 6)
Chimpanzee moved to (9, 1)
Step 3, Time: evening
Orange spreads to (3, 6)
Orange spreads to (2, 4)
Orange spreads to (3, 2)
Orange spreads to (6, 6)
Orangutan at (6, 2) ate Orange
Orangutan moved to (6, 3)
Orangutan reproduced at (7, 3)
Orangutan moved to (9, 2)
Orangutan moved to (9, 8)
Chimpanzee moved to (6, 6)
Orangutan at (8, 4) ate Amanita
Chimpanzee moved to (8, 1)
Orangutan moved to (3, 2)
Orange spread

. O D A D . . O O .
. . . D A O A D . .
. . O . . O . . . .
D . . O O O . . . .
D . An . . . . . . .
. . An . . . . . . .
. . D O . . . C . D
A . An . An . O . A .
C . . . . . . D O .
A . . . A D . An . .

. O D A D . . O O .
. . D D A O A D . .
D . O . . O . . . .
D . . O O O . . . .
D . An . . . . . . .
. . An . . . . . . .
. D D O . . . C . D
A . An . An . O . A .
C . . . . . . D O .
A . . . A D . An . .

. O D A D . . O O .
. . D D A O A D . .
D . O . O O . . . .
D . O O O O O . . .
D . An . . . . . . .
. . An . . . . . . .
. D D . . . O C . D
A . An An An . O . A .
C . . . . . . D O .
A . . . . D . An . .

. O D A D . . O O .
. . D D A O A D . .
D . O . O O A . . .
D . O O O O O . . .
D . An . . . . . . .
. . An . . . . . . .
A D D . . . O C . D
A . An An An . O . A A
C . . . . . . D O .
A . . . . D . An . .

. O D A D O . O O .
. . D D A O A D O .
D . . O O O A . . .
D . O O O O O . . .
D . An . . . . . . .
. . An . . . . . . .
A D D . . O O C . D
A . An An An . O . A A
C . . . .